In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

flight_data = pd.read_csv("C:/Users/Emily Shao/Desktop/archive/itineraries.csv")

In [44]:
# MAY NOT BE NECESSARY
# THIS SECTION MAY NOT BE NECESSARY, TRYING TO CREATE CATEGORICAL DATA

from datetime import datetime 

type(flight_data['flightDate'][0])

flight_data['flightDate'][1048500] # '4/17/2022' type: str

def time_buckets(pdseries):
    for date in pdseries:
        date_object = datetime.strptime(date, '%m/%d/%Y')


'6/5/2022'

In [71]:
import numpy as np

type(flight_data['baseFare'][0])

print(flight_data['baseFare'].max())

def data_buckets(pdseries, num_buckets):
    bins = np.linspace(0, 4000, num_buckets + 1)
    labels = np.ceil(bins[1:]).astype(int)

    bucket_labels = pd.cut(pdseries, bins=bins, labels=labels, right=True)

    return bucket_labels



buckets = 25 # 100 400
flight_data['price_buckets'] = data_buckets(flight_data['baseFare'], buckets)

# # Checking if there are NaN values within the code
# print("Column 'A' has NaN values:", flight_data['price_buckets'].isna().any())

# nan_rows = flight_data[flight_data['price_buckets'].isna()]
# print(nan_rows[['price_buckets', 'baseFare']])




# def to_str(input):
#     return str(input)

# flight_data['price_categories'] = flight_data['price_buckets'].apply(to_str)

# print(flight_data['baseFare'],flight_data['price_buckets']) #, flight_data['price_categories'])


3531.16


In [72]:
encoder = OneHotEncoder(sparse=False)
encoded_features = encoder.fit_transform(flight_data[['startingAirport', 'destinationAirport', 'flightDate']])

# X = pd.concat([pd.DataFrame(encoded_features), flight_data[['baseFare']]], axis=1)

X = pd.concat([pd.DataFrame(encoded_features), flight_data[['price_buckets']]], axis=1)
y = X.pop('price_buckets')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

c:\Users\Emily Shao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [76]:
# output trained with 25 buckets and 1000 iterations = accuracy 0.48353
# output trained with 25 buckets and 300 iterations = accuracy 0.48344
# output trained with 25 buckets and 100 iterations = accuracy 0.48285

model = LogisticRegression(max_iter=100)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

c:\Users\Emily Shao\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 0.48285053525022054


In [ ]:
# TODO

import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.barh(feature_names, model.coef_[0])
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Coefficients of Logistic Regression Model')
plt.show()